In [50]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
OUTPUT_ROOT = "output"

In [17]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [18]:
import random
import numpy as np

# Разбиение обучающих данных
# labels_num -  количество меток (от 0 до 9)
# train_len - длина того обучающего множества, которое мы хотим использовать для обучения, оно должно делиться на количество меток
def balancing(full_train_dataset, lables_num, train_len):
  # Длина всего обучающего множества и того обучающего множества, которое мы хотим использовать для обучения
  full_train_len = len(full_train_dataset)
  # Количество данных с одной меткой
  label_group_num = int(train_len/lables_num)

  # Создаём группы для хранения индексов каждой метки в обучающем наборе данных
  label_groups_index = [[] for _ in range(lables_num)]
  for i in range(full_train_len):
    label = full_train_dataset[i][1]
    label_groups_index[label].append(i)

  # Обрезаем группы, оставляя случайные, неповторяющиеся элементы в каждой и объединяем их всех в один набор индексов
  all_index = np.array([], dtype=int)
  for i in range(lables_num):
    all_index = np.append(all_index, random.sample(label_groups_index[i], label_group_num))
  np.random.shuffle(all_index)

  # Формируем обучающий набор данных
  train_dataset = torch.utils.data.Subset(full_train_dataset, all_index)
  return train_dataset

In [99]:
# Определение модели
# neurons_num - количество нейронов в каждом слое
# img_size - размер изображений из MNIST
class Net(nn.Module):
    def __init__(self, neurons_num, img_size):
        super(Net, self).__init__()
        self.img_size = img_size
        self.fc1 = nn.Linear(img_size, neurons_num)
        self.fc2 = nn.Linear(neurons_num, neurons_num)
        self.fc3 = nn.Linear(neurons_num, neurons_num)
        self.fc4 = nn.Linear(neurons_num, 10)

    def forward(self, x):
        x = x.view(-1, self.img_size)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = nn.functional.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [37]:
# обучение в течение одной эпохи
def train_step(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # вывод текущего прогресса, для того, чтобы убедиться, что обучение идёт
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [68]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    return 100*correct, test_loss

In [93]:
import time
import numpy as np
import os
def train(model, epochs, train_dataloader, test_dataloader, optimizer, loss_fn, output_path):
    train_results = []
    test_results = []
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        startTime = time.time()
        train_step(train_dataloader, model, loss_fn, optimizer)
        print('learning took {:.2f} s'.format(time.time() - startTime))
        train_accuracy, train_loss = test(train_dataloader, model, loss_fn)
        train_results.append((train_accuracy, train_loss))
        test_accuracy, test_loss = test(test_dataloader, model, loss_fn)
        test_results.append((test_accuracy, test_loss))
    os.makedirs(output_path, exist_ok=True)
    np.savetxt(os.path.join(OUTPUT_ROOT, 'train.txt'), np.array(train_results), fmt='%.1f %.8f')
    np.savetxt(os.path.join(OUTPUT_ROOT, 'test.txt'), np.array(test_results), fmt='%.1f %.8f')
    print("Done!")

In [95]:
# Загрузка данных MNIST
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

In [7]:
train_dataset = balancing(train_dataset, 10, 30000)

In [96]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [101]:
# Инициализация модели 
model = Net(1000, 28*28).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [102]:
train(
      model=model, 
      epochs=20, 
      test_dataloader=test_dataloader, 
      train_dataloader=train_dataloader, 
      optimizer=optimizer, 
      loss_fn=loss_fn,
      output_path=OUTPUT_ROOT
      )

Epoch 1
-------------------------------
loss: 2.301903  [   64/60000]
loss: 1.005692  [ 6464/60000]
loss: 0.479035  [12864/60000]
loss: 0.403867  [19264/60000]
loss: 0.265005  [25664/60000]
loss: 0.304077  [32064/60000]
loss: 0.208564  [38464/60000]
loss: 0.278256  [44864/60000]
loss: 0.194847  [51264/60000]
loss: 0.251560  [57664/60000]
learning took 13.10 s
Epoch 2
-------------------------------
loss: 0.101589  [   64/60000]
loss: 0.152749  [ 6464/60000]
loss: 0.125549  [12864/60000]
loss: 0.249332  [19264/60000]
loss: 0.083401  [25664/60000]
loss: 0.207452  [32064/60000]
loss: 0.101985  [38464/60000]
loss: 0.207280  [44864/60000]
loss: 0.134418  [51264/60000]
loss: 0.179388  [57664/60000]
learning took 12.34 s
Epoch 3
-------------------------------
loss: 0.049266  [   64/60000]
loss: 0.102432  [ 6464/60000]
loss: 0.107608  [12864/60000]
loss: 0.174276  [19264/60000]
loss: 0.045040  [25664/60000]
loss: 0.112033  [32064/60000]
loss: 0.081374  [38464/60000]
loss: 0.150898  [44864/600

In [87]:
a = ((1, 2), (3, 4))
np.savetxt("test.txt", np.array(a), fmt='%d %f')